In [1]:
import pandas as pd
import numpy as np

### 2. Carregamento dos Dados

Carregamos o arquivo CSV bruto (`desafio_indicium_imdb.csv`) para um DataFrame do pandas para iniciar a análise e a limpeza.

In [8]:
path = '../data/raw/desafio_indicium_imdb.csv'
df = pd.read_csv(path)
df.head()

### 3. Remoção de Coluna Desnecessária

A coluna `Unnamed: 0` é removida, pois parece ser um índice antigo que foi salvo no arquivo CSV e não tem valor para a análise.

In [11]:
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace= True)

### 4. Análise Inicial e Valores Nulos

Utilizamos `.info()` para ver os tipos de dados de cada coluna e `.isnull().sum()` para contar a quantidade de valores ausentes. Isso nos ajuda a identificar quais colunas precisam de tratamento.

In [20]:
display(df.info())
print('-'*25)
display(df.isnull().sum().sort_values(ascending=False))

### 5. Limpeza da Coluna 'Gross'

A coluna `Gross` (faturamento) está como texto (`object`) por causa das vírgulas. Removemos as vírgulas e a convertemos para um tipo numérico para que possamos usá-la em cálculos.

In [21]:
if df['Gross'].dtype == object:
     df['Gross'] = pd.to_numeric(df['Gross'].str.replace(',',''))

### 6. Verificação de Dados Duplicados

Este loop verifica cada coluna para ver quantos valores duplicados existem. É um passo importante para entender a unicidade dos dados.

In [22]:
for i in df.columns:
    print(f"Column {i} has {df[i].duplicated().sum()} duplicated values")

### 7. Investigação de Duplicados em 'No_of_Votes'

Analisamos especificamente as linhas onde o número de votos (`No_of_Votes`) é duplicado para entender se são erros ou coincidências.

In [23]:
df[df['No_of_Votes'].duplicated(keep=False)].sort_values(by='No_of_Votes')


### 8. Limpeza da Coluna 'Runtime'

A coluna `Runtime` (duração) contém o texto "min". Removemos o texto e convertemos a coluna para um tipo numérico para permitir cálculos.

In [24]:
if df['Runtime'].dtype == 'object':
    df['Runtime'] = df['Runtime'].apply(lambda x: x.split(' ')[0] if isinstance(x, str) else x)
    df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')

df['Runtime'].isna().sum()

### 9. Limpeza da Coluna 'Released_Year'

Identificamos valores não numéricos na coluna de ano de lançamento. Em seguida, corrigimos o valor anômalo (`PG`) para um ano (`1995`) e garantimos que toda a coluna seja do tipo numérico.

In [26]:
estranhos = []
for i in df['Released_Year']:
    if not str(i).isnumeric():
        estranhos.append(i)
        
print(estranhos)

In [27]:
if df['Released_Year'].dtype == object:
    
    df.loc[df['Released_Year'] == 'PG', 'Released_Year'] = 1995
    df['Released_Year'] = pd.to_numeric(df['Released_Year'])

### 10. Análise de Correlação

Calculamos a matriz de correlação das colunas numéricas. Isso ajuda a entender como as variáveis se relacionam entre si (por exemplo, se o faturamento tende a aumentar com a nota do IMDB).

In [33]:
df.corr(numeric_only=True)

### 11. Salvando o DataFrame Limpo

Após a limpeza, salvamos o DataFrame resultante em um novo arquivo CSV na pasta `processed`. Este arquivo será usado nas próximas etapas da análise, como a engenharia de features ou a modelagem.

In [35]:
df.to_csv('../data/processed/pre_feature_eng.csv', index=False)